# 14.TRPO （Trust Region Policy Optimization）：信任区域策略优化
> 即使 **Actor-Critic** 在一定程度上能够降低训练过程中的方差，在实际应用过程中仍与传统策略梯度算法一样存在着 **训练不稳定** 的情况。根本原因在于策略更新过大：网络参数对样本传递的环境信息 **“过度拟合”**，导致训练时全局策略变差，造成训练的不稳定。特别是当面临情况复杂，策略网络是深度模型时，沿着策略梯度更新参数，步长太长，在一段时间内陷入对部分样本的过分信任，会导致全局策略突然显著变差。

> **TRPO（Trust Region Policy Optimization）算法** 最初由 John Schulman 等人于2015年在 [Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477) 中提出。
> 论文引入了 **信任区域（trust region）和KL散度约束** 的概念。在 **信任区域** 上更新策略时能够得到某种策略性能的安全性保证，是其主要思想。**TRPO** 描述了一种优化策略的迭代过程：在理论上能够保证策略学习的 **性能单调性**，并在实际应用中取得了比策略梯度算法更好的效果。

## 14.1 TRPO原理

### 优化目标推导：
> 由于无论策略为什么，采样一条轨迹的**初始状态$s_0$的分布$p(s_0)$固定和策略$\pi$无关**（实际实验中也规定如此），也就是说:
$$\begin{aligned}
J(\theta) & =\mathbb{E}_{\pi}[V^{\pi_\theta}(s_0)] =\sum_{s_0}p(s_0)V^{\pi_\theta}(s_0) \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}[V^{\pi_\theta}(s_0)] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tV^{\pi_\theta}(s_t)-\sum_{t=1}^\infty\gamma^tV^{\pi_\theta}(s_t)\right] \\
 & =-\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty}\gamma^{t}\left(\gamma V^{\pi_{\theta}}(s_{t+1})-V^{\pi_{\theta}}(s_{t})\right)\right]
\end{aligned}$$

> 基于以上等式，可以推导新旧策略的目标函数之间的差距：
$$\begin{aligned}
J(\theta^{\prime})-J(\theta) & =\mathbb{E}_{\pi}\left[V^{\pi_{\theta^{\prime}}}(s_0)\right]-\mathbb{E}_{\pi}\left[V^{\pi_\theta}(s_0)\right] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tr(s_t,a_t)\right]+\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^t\left(\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t)\right)\right] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^t\left[r(s_t,a_t)+\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t)\right]\right]
\end{aligned}$$


> 可见上述公式存在 **时序差分残差** 形式，以下转换成**优势函数A**：
$$J(\theta^{\prime})-J(\theta)=\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tA^{\pi_\theta}(s_t,a_t)\right]$$

> 展开期望形式：
$$J(\theta^{\prime})-J(\theta)=\sum_\tau\left[p(\tau|\pi_{\theta^{\prime}})\sum_{t=0}^\infty\gamma^tA^{\pi_\theta}(s_t,a_t)\right]$$
- 轨迹概率：$p(\tau|\theta)=p(s_0)\prod_{t=0}^{T}[\pi_\theta(a_t|s_t)p(s_{t+1}|s_t,a_t)]$ 
> 又状态访问分布:$\nu^\pi(s)=(1-\gamma)\sum_{t=0}^\infty\gamma^tP(s_t = s, a_t = a \| \pi)$，转换为状态访问分布表达形式：
$$
J(\theta^{\prime})-J(\theta) = \frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta'}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right]
$$

> 所以只要保证:
$$\sum_{s} \left[ \nu^{\pi_{\theta'}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right] \geq 0$$
> 就能保证策略性能的单调递增。

*但是把所有可能的**新策略**都拿来采样数据去**获取状态访问分布**，然后判断哪个**新策略**满足上述条件的做法显然是**不现实**的*
> **TRPO** 做了一步近似操作，**忽略**两个策略之间的状态访问分布变化，直接**采用旧的策略的状态分布$\nu^{\pi_{\theta}}(s)$**：
$$J(\theta^{\prime})-J(\theta) = \frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right]$$

- 当新旧策略非常接近时，状态访问分布变化很小，这么近似是合理的
> 再定义**优化目标**为:
$$\begin{aligned}
L_\theta(\theta^{\prime})& =J(\theta)+\frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right] \\
& =J(\theta)+\frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta}}(s) \sum_{a}\pi_{\theta}(a | s) \left[    \frac{\pi_{\theta^{\prime}}(a|s)}{\pi_\theta(a|s)}A^{\pi_\theta}(s,a) \right] \right] \\
& =J(\theta)+\mathbb{E}_{s\sim\nu^{\pi_\theta}}\mathbb{E}_{a\sim\pi_\theta(\cdot|s)}\left[\frac{\pi_{\theta^{\prime}}(a|s)}{\pi_\theta(a|s)}A^{\pi_\theta}(s,a)\right]
\end{aligned}$$